# Laboratorio 6.2

In [42]:
import nltk
#nltk.download('punkt')
from nltk.corpus import stopwords
#from nlkt.corpus.reader import lin
from nltk.stem import SnowballStemmer
from nltk.util import pr

## Initialize

In [43]:
stoplist = stopwords.words('spanish')
stoplist += ['.','?','-','«', '»', ',', '(', ')', ':', ';'] # No quita todas las comas
stemmer = SnowballStemmer('spanish')
index = {}

### Read files, Tokenize, Stem and Add to index

In [44]:
for i in range (1,7):
    with open("docs/libro{}.txt".format(i), encoding='utf-8') as file:
        text = ""
        lines = file.readlines()
        for line in lines:
            text += line.lower().strip()

        # Tokenize
        words = nltk.word_tokenize(text)

        # Filter
        for token in words:
            if token in stoplist:
                words.remove(token)
        
        # Stem
        for j in range (len(words)):
            words[j] = stemmer.stem(words[j])
        
        # Add to index
        for token in words:
            if token in index.keys():
                index[token].add(i)
            else:
                index[token] = set([i])

## ...
Get the 500 most frequent terms

Sort by document frequency

In [45]:
sortedIndex = {key : val for key, val in sorted(index.items(), key = lambda elem : len(elem[1]), reverse=True)}
mostFrequent = {}
count = 0

for key, val in sortedIndex.items():
    mostFrequent[key] = val
    count += 1
    if (count == 500):
        break

### Sort by key

In [46]:
invertedIndex = {key: val for key, val in sorted(mostFrequent.items(), key = lambda elem: elem[0])}

### Write to file

In [47]:
with open("invertedIndex.txt", 'w', encoding='utf-8') as file:
    for key, value in invertedIndex.items():
        line = key
        line += ':'
        for i in value:
            line += str(i)
            line += ','
        line = line[0:len(line)-1] # Trim last ,
        line += '\n'
        file.writelines(line)

# Queries

 Assign file

In [48]:
indexFile = "invertedIndex.txt"

### Parse Function

In [49]:
def parseLine(line):
    word = ""
    docs = []
    i = 0
    while(i < len(line) and line[i] != ':'):
        word += line[i]
        i += 1
    i += 1
    temp = ""
    while(i < len(line) and line[i] != '\n'):
        if (line[i] == ','):
            docs.append(int(temp))
            temp = ""
        else:
            temp += line[i]
        i += 1
    docs.append(int(temp))
    return (word, docs)

### Read Index Function

In [50]:
invertedIndex2 = {}

In [54]:
def readIndex(index = indexFile):
    with open( index ) as file:
        for line in file:
            item = parseLine(line.strip())
            invertedIndex2[item[0]] = item[1]

    for key, value in invertedIndex.items():
        print(key, ':', str(value))

In [55]:
readIndex()

, : {1, 2, 3, 4, 5, 6}
111º : {1}
a : {2, 3, 5, 6}
abismo.apen : {2}
acab : {1, 4, 5, 6}
accidental : {1}
acebed : {2}
acerc : {2}
acompañ : {1, 2, 5, 6}
acontec : {2, 3}
acud : {1, 2, 5, 6}
adel : {1, 5}
adentr : {1}
agu : {2}
al : {1, 2, 3, 4, 5, 6}
alcanz : {2}
alde : {1}
alert : {1, 4, 5}
alli : {1, 2, 5, 6}
ambos : {1, 5, 6}
amig : {1, 2, 3}
amo : {2, 4, 6}
amon : {1, 2}
anduin : {1, 2, 5}
anill : {1, 2, 4, 6}
anillo.d : {2}
aniquil : {2}
ante : {2, 4}
antes : {1}
antigu : {2}
aparec : {2, 4}
aparent : {3, 4}
apart : {2}
apres : {3}
apresur : {3, 5}
aprovech : {3, 6}
aragorn : {1, 2, 3, 5, 6}
armas : {1}
arnor : {1}
arrastr : {1, 2}
arrebat : {1}
arroj : {2, 3, 6}
ascend : {4, 6}
asent : {2}
asfaloth : {1}
asi : {1, 3, 4, 5, 6}
atac : {1, 2, 4, 5, 6}
ataqu : {3, 5}
atrap : {1, 3, 4}
atraves : {2, 4}
aunqu : {4, 6}
ayud : {1, 3, 5, 6}
años : {1}
baj : {4, 5}
balin : {2}
balrog : {2}
barbol : {3, 6}
barc : {2, 5, 6}
batall : {3, 5, 6}
bien : {1}
bilb : {1, 6}
bolson : {1}
bombadil :

In [78]:
def indexAnd(w1, w2):
    result1 = invertedIndex2[w1]
    result2 = invertedIndex2[w2]
    answer = []
    i = 0
    j = 0
    while (i < len(result1) and j < len(result2)):
        if(result1[i] == result2[j]):
            answer.append(result1[i])
            i += 1
            j += 1
        elif result1[i] < result2[j]:
            i += 1
        else:
            j += 1
    return answer

In [62]:
indexAnd("una", "ungol")

[4, 6]

In [79]:
def indexOr(w1, w2):
    result1 = invertedIndex2[w1]
    result2 = invertedIndex2[w2]
    answer = result1 + result2
    answer = set(answer)
    return answer

In [66]:
indexOr("una","un")

{1, 2, 3, 4, 5, 6}

In [82]:
def indexAndNot(w1, w2):
    result1 = invertedIndex2[w1]
    result2 = invertedIndex2[w2]
    answer = []
    i = 0
    j = 0
    while (i < len(result1) and j < len(result2)):
        if(result1[i] < result2[j]):
            answer.append(result1[i])
            i += 1
        elif(result1[i] == result2[j]):
            i += 1
            j += 1
        elif(result1[i] > result2[j]):
            j += 1
    return answer

In [83]:
indexAndNot("vari","viv")

[]